<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Benjamin Hernth   
- Nombre de alumno 2: Antonia Villegas


### **Link de repositorio de GitHub:** `https://github.com/morbul/MDS7202_entregas`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [2]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow.

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados.
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
# Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [4]:
import codecs

In [5]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text'])
df_comics = df_comics.drop(columns=['Unnamed: 0','name', 'real_name', 'full_name','powers_text']) # eliminar ejemplos sin historia
df_comics_no_label = df_comics_no_label.drop(columns=['Unnamed: 0','name', 'real_name', 'full_name','powers_text'])
df_comics.replace('nan', np.nan, inplace=True)

In [6]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head(1)  

,overall_score,history_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,6,"Delroy Garrett, Jr. grew up to become a track ...",85,30,60,60,40,70,"['Super Speed', 'Super Strength']",[],...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [7]:
df_comics.shape 

(1285, 77)

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset.

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen.

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?.

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [8]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [10]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [11]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [12]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados.

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [13]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [14]:
df_comics['history_text'].tolist()

["Delroy Garrett, Jr. grew up to become a track star and competed in the Olympic Games. After he tested positive for steroids, he lost his three gold medals. In despair, he turned to religion, specifically the Triune Understanding. This group's founder, Jonathan Tremont, had found one of three fragments of a mysterious object and used it to give Garrett the powers of 3-D Man. Garrett assumed that his powers were because of his newfound spiritual enlightenment, and Tremont never disabused him of the notion.",
 ' Richard "Rick" Jones was orphaned at a young age, and after being expelled from several orphanages for disciplinary reasons he was placed into a state institution called Tempest Town. A troubled and rebellious youth, Jones soon came to the attention of the institution\'s chief administrator who smashed his guitar, a gift from his late father, and then had him severely thrashed. Soon afterward, Jones ran away from the institution. He spent the first half of his teens drifting fro

In [15]:
bow2 = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))

history_texts = df_comics['history_text'].tolist()
df2 = bow2.fit_transform(history_texts)

pd.DataFrame(df2.toarray(), columns=bow2.get_feature_names_out())

,!,! !,! '',! (,! ),"! ,",! --,! .,! ``,! alison,...,�kick� .,�kick� driven,�polic,�polic man�,�white,�white room�,�ǣmortal,�ǣmortal instruments���,��wors,"��wors ,"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

In [17]:
import re
# Función para limpiar el texto
def clean_text(text):
    # Remover caracteres especiales y no alfanuméricos
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return cleaned_text

# Aplicar la limpieza del texto a la columna 'history_text'
df_comics['history_text'] = df_comics['history_text'].apply(clean_text)

In [18]:
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

history2 = bog.fit_transform(df_comics["history_text"])

pd.DataFrame(history2.toarray(), columns=bog.get_feature_names_out())

,0,0 dust,002,002 tabitha,0068,0068 hitman,01,01 also,01 zero,03,...,zzzax shoot,zzzax shortcircuit,zzzax sinc,zzzax space,zzzax stand,zzzax still,zzzax submerg,zzzax tri,zzzax water,zzzax zzzax
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,0,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

ocolumns = ['overall_score', 'superpowers', 'alter_egos', 'aliases',
       'place_of_birth', 'first_appearance', 'creator',
       'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race',
       'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img']

preprocessor = ColumnTransformer(
    transformers=[
        ("count_vectorizer", bog, "history_text"),
        ("min_max_scaler", MinMaxScaler(), atributos_de_interes)
          # Agrega tus características aquí
    ],
    remainder="passthrough"
)


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`.

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [20]:
# Obtener todas las columnas de tipo "object"
object_columns = df_comics.select_dtypes(include=['object']).columns
object_columns

Index(['overall_score', 'history_text', 'superpowers', 'alter_egos', 'aliases',
       'place_of_birth', 'first_appearance', 'creator', 'alignment',
       'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race',
       'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img'],
      dtype='object')

In [21]:
num_col = df_comics.select_dtypes(include=np.number).columns
num_col.dtype

dtype('O')

In [22]:
df_comics[num_col] = df_comics[num_col].fillna(0)
print(df_comics[num_col].isna().sum())

intelligence_score                  0
strength_score                      0
speed_score                         0
durability_score                    0
power_score                         0
combat_score                        0
has_electrokinesis                  0
has_energy_constructs               0
has_mind_control_resistance         0
has_matter_manipulation             0
has_telepathy_resistance            0
has_mind_control                    0
has_enhanced_hearing                0
has_dimensional_travel              0
has_element_control                 0
has_size_changing                   0
has_fire_resistance                 0
has_fire_control                    0
has_dexterity                       0
has_reality_warping                 0
has_illusions                       0
has_energy_beams                    0
has_peak_human_condition            0
has_shapeshifting                   0
has_heat_resistance                 0
has_jump                            0
has_self-sus

In [23]:
## Definir el Pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", MultinomialNB())
])
Y = df_comics.loc[:,'alignment'] 
X = df_comics.drop(columns=['overall_score', 'superpowers', 'alter_egos', 'aliases',
       'place_of_birth', 'first_appearance', 'creator', 'alignment',
       'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race',
       'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img'])


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)



In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.62      0.09      0.16        85
        Good       0.58      0.98      0.73       144
     Neutral       0.00      0.00      0.00        28

    accuracy                           0.58       257
   macro avg       0.40      0.36      0.30       257
weighted avg       0.53      0.58      0.46       257



c:\Users\benja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\benja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\benja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


```
Escriba aquí su justificación
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [25]:
pipeline.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('count_vectorizer',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x00000203FF1AF1C8>),
                                    'history_text'),
                                   ('min_max_scaler', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('model', MultinomialNB())],
 'verbose': False,
 'preprocessor': ColumnTransformer(remainder='passthrough',
                   transformers=[('count_vectorizer',
                                  CountVectorizer(ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerToke

In [ ]:

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

'''

# Definimos los clasificadores y los hiperparámetros a ajustar
classifiers = [
    {'classifier': SVC(),
     'params': {'C': [1, 10, 100], 'kernel': ['linear', 'rbf']}},
    {'classifier': RandomForestClassifier(),
     'params': {'n_estimators': [10, 100, 1000], 'max_depth': [None, 5, 10]}},
    {'classifier': KNeighborsClassifier(),
     'params': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}}
]
param_grid = [
    {"Selection__percentile": range(20, 100, 20)},
    {"countvectorizer__max_df": [0.25,0.75,1]},
    {"countvectorizer__min_df": [0.25,0.75,1]},
    
]

for classifier in classifiers:
    halving_pipeline  = HalvingGridSearchCV(
        estimator=classifier['classifier'],
        param_grid=classifier['params'],
        cv=5,
        refit=True,
        n_jobs=-1
    )
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', halving_pipeline )
    ])
    pipeline.fit(x_train, y_train)
    
    ## Imprimimos los resultados
    print("Mejores parámetros encontrados para {}:".format(classifier['classifier'].__class__.__name__))
    print(pipeline.named_steps['classifier'].best_params_)
    print("Exactitud en el conjunto de prueba: {:.2f}".format(pipeline.score(x_test, y_test)))
    print(classification_report(y_test, pipeline.predict(x_test)))


In [29]:
pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("selection", SelectPercentile()),
        ("model", SVC()),
    ]
)


grid = [
    # grilla 1: tree
    {   
        "selection__percentile": range(10, 101, 10),
        "model": [SVC()],
        'model__C': [1, 10, 100],
        'model__kernel': ['linear', 'rbf'], 
    },
    # grilla 2: knn
    {
        "selection__percentile": range(10, 101, 10),
        "model": [KNeighborsClassifier()],
        "model__n_neighbors": [2, 4, 5, 10],
    },
    # grilla 3: random forest
    {
        "model": [RandomForestClassifier()],
        "model__criterion": ["gini", "entropy"],
        "model__bootstrap": [True, False],
    },
]

gs2 = HalvingGridSearchCV(pipe, grid, n_jobs=-1,cv=5,verbose= 10).fit(x_train, y_train)
gs2.best_score_

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 104
n_resources: 30
Fitting 5 folds for each of 104 candidates, totalling 520 fits
----------
iter: 1
n_candidates: 35
n_resources: 90
Fitting 5 folds for each of 35 candidates, totalling 175 fits
----------
iter: 2
n_candidates: 12
n_resources: 270
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 810
Fitting 5 folds for each of 4 candidates, totalling 20 fits


0.653469825933594

In [43]:

# Definimos los clasificadores y los hiperparámetros a ajustar
grid = [
    # grilla 1: tree
    {   
        "selection__percentile": range(10, 101, 10),
        "model": [SVC()],
        'model__C': [1, 10, 100],
        'model__kernel': ['linear', 'rbf'], 
    },
    # grilla 2: knn
    {
        "selection__percentile": range(10, 101, 10),
        "model": [KNeighborsClassifier()],
        "model__n_neighbors": [2, 4, 5, 10],
    },
    # grilla 3: random forest
    {
        "model": [RandomForestClassifier()],
        "model__criterion": ["gini", "entropy"],
        "model__bootstrap": [True, False],
    },
]

pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("selection", SelectPercentile()),
        ("model", SVC()),
    ]
)

for GRID in grid:
    halving_pipeline  = HalvingGridSearchCV(
        pipe,
        GRID,
        cv=1,
        refit=True,
        n_jobs=-1
    )
    halving_pipeline.fit(x_train, y_train)
    
    ## Imprimimos los resultados
    print("Mejores parámetros encontrados para {}:".format(GRID['model'].__class__.__name__))
    print(halving_pipeline.best_params_)
    print("Exactitud en el conjunto de prueba: {:.2f}".format(halving_pipeline.score(x_test, y_test)))
    print(classification_report(y_test, halving_pipeline.predict(x_test)))


Mejores parámetros encontrados para list:
{'model': SVC(C=100), 'model__C': 100, 'model__kernel': 'rbf', 'selection__percentile': 100}
Exactitud en el conjunto de prueba: 0.61
              precision    recall  f1-score   support

         Bad       0.55      0.48      0.51        85
        Good       0.65      0.81      0.72       144
     Neutral       0.25      0.04      0.06        28

    accuracy                           0.61       257
   macro avg       0.48      0.44      0.43       257
weighted avg       0.57      0.61      0.58       257

Mejores parámetros encontrados para list:
{'model': KNeighborsClassifier(n_neighbors=10), 'model__n_neighbors': 10, 'selection__percentile': 50}
Exactitud en el conjunto de prueba: 0.56


c:\Users\benja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\benja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\benja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         Bad       0.44      0.05      0.09        85
        Good       0.56      0.97      0.71       144
     Neutral       0.00      0.00      0.00        28

    accuracy                           0.56       257
   macro avg       0.34      0.34      0.27       257
weighted avg       0.46      0.56      0.43       257

Mejores parámetros encontrados para list:
{'model': RandomForestClassifier(bootstrap=False), 'model__bootstrap': False, 'model__criterion': 'gini'}
Exactitud en el conjunto de prueba: 0.61
              precision    recall  f1-score   support

         Bad       0.61      0.26      0.36        85
        Good       0.61      0.93      0.74       144
     Neutral       0.00      0.00      0.00        28

    accuracy                           0.61       257
   macro avg       0.41      0.40      0.37       257
weighted avg       0.54      0.61      0.53       257



In [1]:
# Obtenemos el mejor modelo y los mejores hiperparámetros
best_model = gs2.best_estimator_
best_params = gs2.best_params_

# Imprimimos los resultados
print("Mejor modelo encontrado:")
print(best_model)
print("Mejores hiperparámetros encontrados:")
print(best_params)

# Realizamos la predicción y obtenemos el informe de clasificación en el conjunto de prueba
y_pred = best_model.predict(x_test)
classification_rep = classification_report(y_test, y_pred)
print("Informe de clasificación:")
print(classification_rep)

NameError: name 'gs2' is not defined

```
Escriba aquí su justificación
```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [ ]:
#### Código aquí ####

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>